<a href="https://colab.research.google.com/github/Hamzakhan942/Twitter-Analysis-VSM/blob/main/Twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Twitter Analysis by similairty measures

Adding the API

In [ ]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
!pip3 install --upgrade aiohttp_socks 

In [3]:
# Import Library
import twint

Scrapping Trump's Tweets

In [4]:
# Instantiate and configure the twint-object
c = twint.Config()

import nest_asyncio
nest_asyncio.apply()

c.Store_object = True
c.Pandas =True
c.Search = "realDonaldTrump"
c.Limit = 10000
c.Lang = 'en'

In [5]:
# Run search
twint.run.Search(c)

Streaming output truncated to the last 5000 lines.
1341667293198249984 2020-12-23 08:49:44 +0000 <Cathy_Donelson> @pillowmilk @Melanie41171588 @realDonaldTrump LOL. Good one!
1341667292988502017 2020-12-23 08:49:44 +0000 <JuleBeachboard> @SenRonJohnson @realDonaldTrump @jbinnall  https://t.co/AuIvJ15cjB
1341667292745183238 2020-12-23 08:49:44 +0000 <DrBobGames> @SidneyPowell1 @realDonaldTrump @GOP @GenFlynn @JosephJFlynn1 @LLinWood @PatrickByrne Screw the GOP new party now.
1341667291474391047 2020-12-23 08:49:43 +0000 <Putin7111952> @McKaylaRoseJ @funder @realDonaldTrump You can Shere taxes between from 300k dead Americans.  Then all country go to lockdown he start to do mega rallys to spread virus across the country.
1341667290337730560 2020-12-23 08:49:43 +0000 <XSenJ11> @RichardBSpencer @VDAREJamesK @realDonaldTrump Waaaaaaay ahead of Trump   If VDare cared about journalistic integrity they'd note Nancy Pelosi was ahead of the curve
1341667285883367426 2020-12-23 08:49:42 +0000 <Ra

In [6]:
# Quick check
twint.storage.panda.Tweets_df

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1341674128835633152,1341653694736027648,1.608715e+12,2020-12-23 09:16:54,+0000,,@vanOnselenP @ozrock67 @GladysB is channeling ...,en,[],[],35437635,35437635,godemons27,Demonforce🌊🌊🌊#FBR,3,09,https://twitter.com/godemons27/status/13416741...,[],[],0,,False,0,0,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'vanOnselenP', 'name': 'Peter...",,,,
1,1341674128147849222,1340333618691002368,1.608715e+12,2020-12-23 09:16:53,+0000,,@MatthewJshow @realDonaldTrump https://t.co/V...,und,[],[],1245837644828819456,1245837644828819456,HegemonOzz,OZZHegemon,3,09,https://twitter.com/HegemonOzz/status/13416741...,[],[https://pbs.twimg.com/media/Ep6VgEqUcAA2yAg.jpg],1,https://pbs.twimg.com/media/Ep6VgEqUcAA2yAg.jpg,False,0,0,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'MatthewJshow', 'name': 'Matt...",,,,
2,1341674126298116096,1341064514469580801,1.608715e+12,2020-12-23 09:16:53,+0000,,@strangebird315 @redgwell_susan @tracybeanz @r...,en,[],[],717124010,717124010,prutang840,kyle,3,09,https://twitter.com/prutang840/status/13416741...,[],[],0,,False,0,0,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'strangebird315', 'name': 'St...",,,,
3,1341674124406476800,1341405487057821698,1.608715e+12,2020-12-23 09:16:53,+0000,,@realDonaldTrump https://t.co/e9XojwctxG,und,[],[],1290248042474434560,1290248042474434560,qaAzw0mwcZuf8xD,香澄汁,3,09,https://twitter.com/qaAzw0mwcZuf8xD/status/134...,[],[https://pbs.twimg.com/media/Ep6VaPNVQAAmKF5.jpg],1,https://pbs.twimg.com/media/Ep6VaPNVQAAmKF5.jpg,False,0,0,0,,realDonaldTrump,,,,,,,[],,,,
4,1341674122636480512,1341636082207780864,1.608715e+12,2020-12-23 09:16:52,+0000,,@Lincoln22873835 @SidneyPowell1 @realDonaldTru...,en,[],[],1256088954685673472,1256088954685673472,Slipperyslopes6,Casper-Good people are more valuable than the ...,3,09,https://twitter.com/Slipperyslopes6/status/134...,[https://youtu.be/H9ng206C0wI],[],0,,False,0,0,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'Lincoln22873835', 'name': 'L...",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1341661035833520129,1341629778596667395,1.608712e+12,2020-12-23 08:24:52,+0000,,@JackPosobiec @realDonaldTrump needs to fire e...,en,[],[],2443251613,2443251613,OnTheRocksSteph,StephOnTheRocks,3,08,https://twitter.com/OnTheRocksSteph/status/134...,[],[],0,,False,0,0,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'JackPosobiec', 'name': 'Jack...",,,,
9996,1341661035602833413,1341537886315950080,1.608712e+12,2020-12-23 08:24:52,+0000,,@MontyVersace1 @realDonaldTrump 100% correct!,en,[],[],34474967,34474967,CrazyCatMom42,Trump is an American Traitor,3,08,https://twitter.com/CrazyCatMom42/status/13416...,[],[],0,,False,0,0,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'MontyVersace1', 'name': 'Mon...",,,,
9997,1341661034881429504,1341537886315950080,1.608712e+12,2020-12-23 08:24:52,+0000,,@mbombino @fuck__djt @realDonaldTrump Yes the ...,en,[],[],206861151,206861151,bmance1,Bradford Mance,3,08,https://twitter.com/bmance1/status/13416610348...,[],[],0,,False,0,3,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'mbombino', 'name': 'Weight l...",,,,
9998,1341661033937727488,1341537886315950080,1.608712e+12,2020-12-23 08:24:52,+0000,,@realDonaldTrump Don’t back down @realDonaldTr...,en,[],[],318450747,318450747,RobynLudtke1,Robyn Ludtke,3,08,https://twitter.com/RobynLudtke1/status/134166...,[],[],0,,False,1,2,0,,realDonaldTrump,,,,,,,"[{'screen_name': 'realDonaldTrump', 'name': 'D...",,,,


Saving Data set to a text file

In [7]:
data_set = open('Tweets-Dataset-trump.txt', 'w')
data_set.writelines('---')
data_set.writelines('\n')
for index, row in enumerate(twint.storage.panda.Tweets_df.values):
  # print(index, row[0], row[3], row[6])
  data_set.writelines(str(index))
  data_set.writelines('\n')
  data_set.writelines(row[0])
  data_set.writelines('\n')
  data_set.writelines(row[3])
  data_set.writelines('\n')
  data_set.writelines(row[6])
  data_set.writelines('\n')
  data_set.writelines('---')
  data_set.writelines('\n')

In [8]:
data_set.close()

Reading our dataset text file

In [9]:
data_dict = {}
tweet_id = []
tweet_date = []
tweet = []


new_d = open('Tweets-Dataset-trump.txt', 'r')
set_data = new_d.read().split('---')
for data in set_data[:200]:
  # print(data.split('\n'))
  if len(data.split('\n')) > 3:
    tweet_id.append(data.split('\n')[2])
    tweet_date.append(data.split('\n')[3])
    tweet.append(data.split('\n')[4])

data_dict['tweet_id'] = tweet_id
data_dict['tweet_data'] = tweet_date
data_dict['tweet'] = tweet


In [10]:
print(data_dict)

{'tweet_id': ['1341674128835633152', '1341674128147849222', '1341674126298116096', '1341674124406476800', '1341674122636480512', '1341674121034346497', '1341674120115875843', '1341674118530396160', '1341674115816669185', '1341674114654810114', '1341674112586936322', '1341674112188596225', '1341674111311904768', '1341674109038522369', '1341674108904480768', '1341674108229210112', '1341674106543026180', '1341674106459058177', '1341674104366256128', '1341674104194297857', '1341674103632240640', '1341674103577608192', '1341674102856118272', '1341674102105513985', '1341674102025760768', '1341674101488828419', '1341674100859654146', '1341674099744108544', '1341674099328778240', '1341674098666180609', '1341674097395159040', '1341674095851810816', '1341674095793082368', '1341674095407149057', '1341674095226867713', '1341674094819831808', '1341674094752903168', '1341674094119559168', '1341674091242262529', '1341674085894385664', '1341674084900478976', '1341674084254400513', '1341674084208320512

Dataframes to hold dataset

In [11]:
import pandas as pd

dataset_frame = pd.DataFrame(data=data_dict)
dataset_tweets = pd.DataFrame(data=data_dict)

Module for data cleaning

In [12]:
import re
def remove_urls (tweet):
    tweet = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags=re.MULTILINE)
    return(tweet)

In [13]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Preprocessing Step

In [14]:
raw_tweets = dataset_frame['tweet']
ps = PorterStemmer()

preprocessed_tweets = []
for tweet in raw_tweets:
  tweet = remove_urls(tweet)
  tweet = word_tokenize(tweet)
  tweet = [word for word in tweet if word not in stop_words]
  tweet = [word for word in tweet if word.isalpha()]
  tweet = [ps.stem(word) for word in tweet]
  tweet = [word.lower() for word in tweet]
  preprocessed_tweets.append(tweet)

dataset_frame['tweet'] = preprocessed_tweets

Loss rate of tweets due to blanks created by preprocessing

In [15]:
count = 0
for tweet in dataset_frame['tweet']:
  if len(tweet) > 0:
    count+=1

print(f"Loss Rate: {(200-count)/200 * 100}%")

Loss Rate: 0.5%


In [16]:
dataset_frame 

,tweet_id,tweet_data,tweet
0,1341674128835633152,2020-12-23 09:16:54,"[vanonselenp, gladysb, channel, realdonaldtrum..."
1,1341674128147849222,2020-12-23 09:16:53,"[matthewjshow, realdonaldtrump]"
2,1341674126298116096,2020-12-23 09:16:53,"[tracybeanz, realdonaldtrump, mic]"
3,1341674124406476800,2020-12-23 09:16:53,[realdonaldtrump]
4,1341674122636480512,2020-12-23 09:16:52,"[realdonaldtrump, patrickbyrn, llinwood, major..."
...,...,...,...
194,1341673867438276608,2020-12-23 09:15:51,"[realdonaldtrump, still, go, wow]"
195,1341673865051840513,2020-12-23 09:15:51,[realdonaldtrump]
196,1341673862325563393,2020-12-23 09:15:50,"[realdonaldtrump, newtgingrich, oann, fucker]"
197,1341673862145036290,2020-12-23 09:15:50,"[realdonaldtrump, patrickbyrn, llinwood, major..."


In [17]:
corpus = []

[corpus.append(text) for text in dataset_frame['tweet']]
corpus

[['vanonselenp', 'gladysb', 'channel', 'realdonaldtrump', 'bs'],
 ['matthewjshow', 'realdonaldtrump'],
 ['tracybeanz', 'realdonaldtrump', 'mic'],
 ['realdonaldtrump'],
 ['realdonaldtrump',
  'patrickbyrn',
  'llinwood',
  'majorpatriot',
  'majorkillswitch',
  'mariabartiromo',
  'tomfitton',
  'robertplewi',
  'barbararedg',
  'they',
  'do',
  'serv',
  'us',
  'a',
  'punk',
  'song',
  'assang',
  'get',
  'mention',
  'we',
  'need',
  'free',
  'everi',
  'rt',
  'subscrib',
  'help',
  'improv',
  'qualiti'],
 ['realdonaldtrump', 'let', 'geaux'],
 ['ligonmaximu',
  'smithkish',
  'realdonaldtrump',
  'mayb',
  'two',
  'listen',
  'eh',
  'mayb',
  'learn',
  'word',
  'mostli',
  'media',
  'dead',
  'set',
  'misrepres',
  'point',
  'skew',
  'word',
  'spin'],
 ['argusskyhawk',
  'realdonaldtrump',
  'ye',
  'die',
  'heat',
  'summer',
  'have',
  'learn',
  'anyth',
  'liar',
  'media',
  'govern',
  'ex',
  'faucci',
  'you',
  'know',
  'surviv',
  'rate',
  'peopl',
  '

In [18]:
corpus_sklearn = []

for sentence in dataset_frame['tweet']:
  sent = " ". join(sentence)
  corpus_sklearn.append(sent)

corpus_sklearn

['vanonselenp gladysb channel realdonaldtrump bs',
 'matthewjshow realdonaldtrump',
 'tracybeanz realdonaldtrump mic',
 'realdonaldtrump',
 'realdonaldtrump patrickbyrn llinwood majorpatriot majorkillswitch mariabartiromo tomfitton robertplewi barbararedg they do serv us a punk song assang get mention we need free everi rt subscrib help improv qualiti',
 'realdonaldtrump let geaux',
 'ligonmaximu smithkish realdonaldtrump mayb two listen eh mayb learn word mostli media dead set misrepres point skew word spin',
 'argusskyhawk realdonaldtrump ye die heat summer have learn anyth liar media govern ex faucci you know surviv rate peopl have research death flu pneumonia we cure',
 'speakerpelosi realdonaldtrump ask stand alon relief bill month ago you refus what page wast amp nonsens absolut shame',
 'realdonaldtrump communist china joe want american lock forev',
 'realdonaldtrump do cri like littl liber b biden lose',
 'whitehous realdonaldtrump',
 'jessiprincey realdonaldtrump goodby newsma

In [19]:
count_bow = {}

for sentence in corpus:
  for word in sentence:
    if word not in count_bow:
      count_bow[word] = 0
    else:
      count_bow[word] += 1

In [20]:
len(count_bow)

1267

Filter to reduce dimmensions

In [21]:
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0

for word in count_bow.keys():
  if count_bow[word] > 0:
    count_0 += 1
  if count_bow[word] > 1:
    count_1 += 1
  if count_bow[word] > 2:
    count_2 += 1
  if count_bow[word] > 3:
    count_3 += 1

print(f"Filter Rate: word filter @ 0 = {(len(count_bow) - count_0)/len(count_bow) * 100}")
print(f"Filter Rate: word filter @ 1 = {(len(count_bow) - count_1)/len(count_bow) * 100}")
print(f"Filter Rate: word filter @ 2 = {(len(count_bow) - count_2)/len(count_bow) * 100}")
print(f"Filter Rate: word filter @ 3 = {(len(count_bow) - count_3)/len(count_bow) * 100}")

Filter Rate: word filter @ 0 = 68.19258089976323
Filter Rate: word filter @ 1 = 82.00473559589582
Filter Rate: word filter @ 2 = 88.23993685872139
Filter Rate: word filter @ 3 = 91.63378058405682


In [22]:
vocabulary = []
for word in count_bow.keys():
  if count_bow[word] > 2 and word not in stop_words:
    vocabulary.append(word)
  
len(vocabulary)

120

Creating a count vectorizer

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(vocabulary=vocabulary)
X = vectorizer.fit_transform(corpus_sklearn)

print(vectorizer.get_feature_names())

['realdonaldtrump', 'matthewjshow', 'patrickbyrn', 'llinwood', 'majorpatriot', 'majorkillswitch', 'mariabartiromo', 'tomfitton', 'robertplewi', 'barbararedg', 'us', 'get', 'mention', 'need', 'free', 'everi', 'help', 'let', 'mayb', 'media', 'ye', 'die', 'know', 'peopl', 'death', 'speakerpelosi', 'ask', 'stand', 'relief', 'bill', 'month', 'amp', 'shame', 'china', 'want', 'american', 'like', 'biden', 'go', 'play', 'good', 'done', 'come', 'america', 'state', 'lie', 'lindseygrahamsc', 'covid', 'republican', 'even', 'democrat', 'said', 'trump', 'support', 'twitter', 'sexcounsel', 'problem', 'genflynn', 'potu', 'alway', 'evid', 'never', 'accept', 'time', 'first', 'fight', 'live', 'famili', 'follow', 'vote', 'got', 'pleas', 'presid', 'take', 'elect', 'fraud', 'god', 'back', 'actual', 'think', 'fool', 'work', 'expos', 'war', 'thi', 'u', 'veto', 'loser', 'gop', 'call', 'thank', 'make', 'one', 'year', 'see', 'ignor', 'everyon', 'lost', 'money', 'must', 'corrupt', 'countri', 'day', 'busi', 'gon', 

In [24]:
X.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

A tf-idf vectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary=vocabulary)
X2 = vectorizer.fit_transform(corpus_sklearn)
# print(vectorizer.get_feature_names())

# print(X2.shape)

In [26]:
# len(X2.toarray())
# print(vectorizer.get_feature_names())

Enter a query to test

In [33]:
query = "joe biden rigged the elections"

In [34]:
word = word_tokenize(query)
word = [token for token in word if token not in stop_words]
word = [token for token in word if token.isalpha()]
word = [ps.stem(token) for token in word]
word = [token.lower() for token in word]
query = word
query = " ".join(query)

query

'joe biden rig elect'

Fitting query

In [35]:
vect = TfidfVectorizer(vocabulary=vocabulary)
XQ = vect.fit_transform([query])

# print(vectorizer.get_feature_names())
# print(X2.shape)

Similarity calculation

In [36]:
# print(XQ.toarray())

# sklearn.metrics.pairwise.cosine_similarity(X, Y=None, dense_output=True)
from sklearn.metrics.pairwise import cosine_similarity, paired_cosine_distances

ans = cosine_similarity(XQ.toarray(), X2.toarray())

ans.T.shape

(199, 1)

In [37]:
dataset_frame['similarity'] = ans.T

In [38]:
dataset_frame

,tweet_id,tweet_data,tweet,similarity
0,1341674128835633152,2020-12-23 09:16:54,"[vanonselenp, gladysb, channel, realdonaldtrum...",0.0
1,1341674128147849222,2020-12-23 09:16:53,"[matthewjshow, realdonaldtrump]",0.0
2,1341674126298116096,2020-12-23 09:16:53,"[tracybeanz, realdonaldtrump, mic]",0.0
3,1341674124406476800,2020-12-23 09:16:53,[realdonaldtrump],0.0
4,1341674122636480512,2020-12-23 09:16:52,"[realdonaldtrump, patrickbyrn, llinwood, major...",0.0
...,...,...,...,...
194,1341673867438276608,2020-12-23 09:15:51,"[realdonaldtrump, still, go, wow]",0.0
195,1341673865051840513,2020-12-23 09:15:51,[realdonaldtrump],0.0
196,1341673862325563393,2020-12-23 09:15:50,"[realdonaldtrump, newtgingrich, oann, fucker]",0.0
197,1341673862145036290,2020-12-23 09:15:50,"[realdonaldtrump, patrickbyrn, llinwood, major...",0.0


Sorting by similarity 

In [39]:
dataset_frame = dataset_frame.sort_values(by=['similarity'], ascending=False)
dataset_frame 

,tweet_id,tweet_data,tweet,similarity
10,1341674112586936322,2020-12-23 09:16:50,"[realdonaldtrump, do, cri, like, littl, liber,...",0.529942
166,1341673907338809349,2020-12-23 09:16:01,"[realdonaldtrump, wake, up, stupid, you, lost,...",0.506353
183,1341673888569204737,2020-12-23 09:15:56,"[funder, realdonaldtrump, you, wish, there, po...",0.428374
80,1341674018773078017,2020-12-23 09:16:27,"[realdonaldtrump, wtf, think, biden, make, whi...",0.411471
115,1341673967443165185,2020-12-23 09:16:15,"[evanakilgor, realdonaldtrump, as, parti, cons...",0.338093
...,...,...,...,...
72,1341674035781001216,2020-12-23 09:16:31,"[realdonaldtrump, newtgingrich, oann, you, peo...",0.000000
73,1341674035088945152,2020-12-23 09:16:31,"[lindseygrahamsc, realdonaldtrump, i, think, n...",0.000000
74,1341674034120056833,2020-12-23 09:16:31,"[thi, also, goe, senatemajldr, senjoniernst, l...",0.000000
75,1341674029724233729,2020-12-23 09:16:30,"[santonedward, undoingital, dpwimm, realdonald...",0.000000


Top 3 search results

In [40]:
best_match_id = dataset_frame['tweet_id'][:3].tolist()

Results

In [41]:
for id in best_match_id:
 result_row = dataset_tweets[dataset_tweets['tweet_id'] == id]
 link = 'https://twitter.com/realDonaldTrump/status/'
 link += result_row['tweet_id'].tolist()[0]
 t_tweet = result_row['tweet'].tolist()[0]
 t_date = result_row['tweet_data'].tolist()[0]
 print(f"Tweet posed on: "+t_date)
 print(f"Tweet: "+t_tweet)
 print(f"Click to view tweet: "+link)

 print('----------------------------------------------------')

Tweet posed on: 2020-12-23 09:16:50
Tweet: @RoccoBrown11 @pablo_honey1 @realDonaldTrump Don't cry like a little liberal b when Biden loses.
Click to view tweet: https://twitter.com/realDonaldTrump/status/1341674112586936322
----------------------------------------------------
Tweet posed on: 2020-12-23 09:16:01
Tweet: @realDonaldTrump WAKE UP STUPID!! YOU LOST!!! YOU LOST!!! YOU LOST!!! IT IS OVER!! BIDEN WON! BIDEN WON! BIDEN WON!!! BIDEN WON!!!! DOESN’T MATTER WHAT GINGRICH SAYS! YOU LOST! BIDEN WILL BE INAGURATED ON JAN 20, 2021! AFTER BEING A SORE LOSER, WE’LL NEVER LET YOU BE POTUS AGAIN!
Click to view tweet: https://twitter.com/realDonaldTrump/status/1341673907338809349
----------------------------------------------------
Tweet posed on: 2020-12-23 09:15:56
Tweet: @sinept69 @funder @realDonaldTrump 😂 You wish. There’s no possibility the election will be overturned. There is no evidence of widespread voter fraud that would amount to swinging the election in his favor. It’s over. H